Many diffusion models for protein structure generation require a set of atom coordinates to be diffused with Gaussian noises having a predefined variance schedule, which finally results in a randomized set of coordinates distributed according to 3D Gaussian distribution.
This tutorial shows how to use the `StructureBatch` object to generate a set of diffused coordinates. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import torch

import protstruc as ps
from protstruc.general import ATOM

/data/project/dohoon/miniconda3/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math

def cosine_variance_schedule(T, s=8e-3, beta_max=0.999):
    # cosine variance schedule
    # T: total timesteps
    # s: small offset to prevent beta from being too small
    # beta_max: to prevent singularities at the end of the diffusion process
    t = torch.arange(T + 1)  # 0, 1, ..., T

    f_t = torch.cos((t / T + s) / (1 + s) * math.pi / 2.0).square()
    alpha_bar = f_t / f_t[0]
    beta = torch.cat(
        [
            torch.tensor([0.0]),
            torch.clip(1 - alpha_bar[1:] / alpha_bar[:-1], min=1e-5, max=beta_max),
        ]
    )
    alpha = 1 - beta

    sched = {
        "alpha": alpha,
        "alpha_bar": alpha_bar,
        "alpha_bar_sqrt": alpha_bar.sqrt(),
        "one_minus_alpha_bar_sqrt": (1 - alpha_bar).sqrt(),
        "beta": beta,
    }
    return sched

In [3]:
pdb_id = '4EOT'
sb = ps.StructureBatch.from_pdb_id(pdb_id)
prt_idx = 0
atom_idx = ATOM.CA

fig = plt.figure(figsize=(14, 5))
ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133)

sb.standardize()

T = 1000
sched = cosine_variance_schedule(T=T, s=8e-3, beta_max=0.999)

ims = []
for t in range(T):
    # coordinates of Ca atoms
    # xyz = sb.get_xyz().numpy()
    xyz = sched['alpha_bar_sqrt'][t] * sb.get_xyz()
    bsz, n_res = xyz.shape[:2]

    # add random Gaussian noise to the coordinates 
    xyz += (1 - sched['alpha_bar'][t]) * torch.randn(bsz, n_res, 1, 3)

    xyz = xyz.numpy()
    ax1.set_title(f't={t}')
    im1 = ax1.scatter(
        xyz[prt_idx, :, atom_idx, 0],
        xyz[prt_idx, :, atom_idx, 1],
        xyz[prt_idx, :, atom_idx, 2],
        c='C1'
    )
    ax2.set_title(f't={t}')
    im2, = ax2.plot(
        xyz[prt_idx, :, atom_idx, 0],
        xyz[prt_idx, :, atom_idx, 1],
        xyz[prt_idx, :, atom_idx, 2],
        c='C1'
    )
    # histogram of x coordinates
    _, _, im3 = ax3.hist(xyz[prt_idx, :, atom_idx, 0], bins=33, fc='C1')

    # histogram patches (im3) is already a list, so just concatenate it
    ims.append([im1, im2] + list(im3))

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
ani.save(f'animations/{pdb_id}_diffusion.gif')

plt.clf()  # not showing the results after this cell

<Figure size 1400x500 with 0 Axes>

The animation below shows that the coordinates of Ca atoms gradually reaches to the Gaussian distribution.

![tmp](/tutorials/animations/4EOT_diffusion.gif)